Reading the class label of train data, generating the arrayDocCount array for saving the count of documents for each class, this would give us the prior probability.

In [0]:
trainlabelFile = open('train.label')
labelline = trainlabelFile.readline()

arrayClassCount = [ 0 for i in range(0,21)]
arrayDocCount = [ 0 for i in range(0,21)]
docdict = {}
cnt = 1
while labelline:
    currentlabel = labelline.strip().split()
    classIndex = int(currentlabel[0])
    docdict[cnt] = classIndex
    arrayDocCount[classIndex] += 1
    cnt += 1
    labelline = trainlabelFile.readline()

Reading train data and saving the data in a matrix, where each row represent the count of words across all documents which belongs to one class has seen the words. 

In [0]:
traindataFile = open('train.data')
matrixProbabilityWord = [[1 for j in range(0,61189)] for i in range(0,21) ]
dataline = traindataFile.readline()
while dataline:
    currentdocword = dataline.strip().split()
    docIndex = int(currentdocword[0])
    wordIndex = int(currentdocword[1])
    wordCount = int(currentdocword[2])
    matrixProbabilityWord[docdict[docIndex]][wordIndex] += wordCount
    arrayClassCount[docdict[docIndex]] += wordCount
    dataline = traindataFile.readline()

Calculating the likelihoods of individual events 
(IIDs)

In [0]:
for i in range(1,21):
    for j in range(1,61189):
      matrixProbabilityWord[i][j] /= arrayClassCount[i]

In [0]:
wordDocumentavailability = [[0 for j in range(0,61189)] for i in range(0,7506)]
testdataFile = open('test.data')

Reading the test data to create the boolean matrix. Each row represents the frequency of words.

In [0]:
dataline = testdataFile.readline()
docIndex = 0
while dataline:
    currentdocword = dataline.strip().split()
    docIndex = int(currentdocword[0])
    wordIndex = int(currentdocword[1])
    count = int(currentdocword[2])
    wordDocumentavailability[docIndex][wordIndex] = count
    dataline = testdataFile.readline()

In [0]:
total_docs = 0
for i in range(0,21):
    total_docs += arrayDocCount[i]

Calculating posterior probabilities, we are taking log of individual likelihoods(IIDs) so that we don't miss information.

In [7]:
import math
testlabelFile = open('test.label')
labelline = testlabelFile.readline()
docId = 1
correctmapping = 0
#print(labelline)


while labelline:
    currentline = labelline.strip().split()
    classlabel = int(currentline[0])
    #print(classlabel)
    maxValue = float('-inf')
    prediction = 0
    for c in range(1,21):
        py = 0
        for word in range(1,61189):
        #print(word)
            py += wordDocumentavailability[docId][word]*math.log(matrixProbabilityWord[c][word])
        py += math.log(arrayDocCount[c]/total_docs)
        if py > maxValue:
            prediction = c
            maxValue = py

    #print(docId,prediction)
    #print(prediction)
    if  prediction == classlabel:
        correctmapping+= 1
    labelline = testlabelFile.readline()
    docId += 1

print(correctmapping)
                

5836


In [8]:
print(correctmapping/7505)

0.7776149233844104


The success rate while treating the data as binomial distribution, yields us 77.76 % accuracy.